<a href="https://colab.research.google.com/github/Yassmine2020/EmailAssistant/blob/main/email_assistant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install torch

In [ ]:
!pip install gradio_client
!pip install langdetect
!pip install elevenlabs
!pip install pydantic
!pip install  elevenlabs soundfile gradio_client langdetect
!pip install pydub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 305.1/305.1 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993225 sha256=e188ebe9498477fdbdf6dc4514beab5fb032e873ea34d768904212d077c1e9b6
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 381.9/381.9 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━

In [ ]:
from gradio_client import Client

client = Client("https://facebook-seamless-m4t.hf.space/")

from langdetect import detect
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer
import numpy as np
from scipy.special import softmax
import csv
import urllib.request
import io
from pydub import AudioSegment  # For audio file operations
from pydub.playback import play  # For playing audio
import elevenlabs
from elevenlabs import generate, play
import torch
from transformers import pipeline

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loaded as API: https://facebook-seamless-m4t.hf.space/ ✔


#### Translate to english

In [ ]:
## Translate to english
def arabic_to_eng(text):
    result = client.predict(
        "T2TT (Text to Text translation)",
        "file",
        "",
        "",
        text,
        "Modern Standard Arabic",
        "English",
        api_name="/run"
    )
    return result[1]

def french_to_eng(text):
    result = client.predict(
        "T2TT (Text to Text translation)",
        "file",
        "",
        "",
        text,
        "French",
        "English",
        api_name="/run"
    )
    return result[1]

def translate_to_english(text):
    try:
        language = detect(text)
    except Exception as e:  # Catch a specific exception if possible
        return f"Language detection failed: {e}"

    if language == 'ar':
        return arabic_to_eng(text)
    elif language == 'fr':
        return french_to_eng(text)
    elif language == 'en':  # Corrected language code for English
        return text
    else:
        return "Unsupported language"

In [ ]:
# Example usage
test = 'السلام عليكم'
translate_to_english(test)

'Peace be with you.'

In [ ]:
arabic_to_eng('السلام عليكم')

'Peace be with you.'

In [ ]:
french_to_eng('Bonjour')

'Hello. How are you?'

#### Sentiment analysis

In [ ]:
task='sentiment'
MODEL = f"cardiffnlp/twitter-roberta-base-{task}"

tokenizer = AutoTokenizer.from_pretrained(MODEL)

# download label mapping
labels=[]
mapping_link = f"https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/{task}/mapping.txt"
with urllib.request.urlopen(mapping_link) as f:
    html = f.read().decode('utf-8').split("\n")
    csvreader = csv.reader(html, delimiter='\t')
labels = [row[1] for row in csvreader if len(row) > 1]

# PT
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
model.save_pretrained(MODEL)

OSError: Can't load tokenizer for 'cardiffnlp/twitter-roberta-base-sentiment'. If you were trying to load it from 'https://huggingface.co/models', make sure you don't have a local directory with the same name. Otherwise, make sure 'cardiffnlp/twitter-roberta-base-sentiment' is the correct path to a directory containing all relevant files for a RobertaTokenizerFast tokenizer.

In [ ]:
def text_to_sentiment(eng_text):
  encoded_input = tokenizer(eng_text, return_tensors='pt')
  output = model(**encoded_input)
  scores = output[0][0].detach().numpy()
  scores = softmax(scores)

  # Find the index of the highest score
  max_index = np.argmax(scores)
  sentiment = labels[max_index]
  # score = scores[max_index]   # , np.round(float(score), 4)

  return sentiment

In [ ]:
# Example usage
test = "The warm sunlight gently streamed through the windows, filling the room with a soft, golden glow, as laughter and cheerful conversation echoed amongst friends, creating an atmosphere of joy and contentment that uplifted everyone's spirits"
sentiment = text_to_sentiment(test)
print(f"Sentiment: {sentiment}")

IndexError: list index out of range

#### Text to speech

In [ ]:
def text_to_speech(text):
    # Checking if the Arabic answer is non-empty
    if text:
        # Setting the API key for Eleven Labs TTS service
        elevenlabs.set_api_key("8baca584c9025aa9c7f85e0e4e8ae0c1")
        # Generating audio from the Arabic answer using Eleven Labs TTS
        audio = generate(
            text=text,
            voice="Daniel",  # Choosing the voice for the generated audio
            model='eleven_multilingual_v2'  # Choosing the TTS model
        )
    else:
        print("▶️ empty ar_answer")

    # Converting the generated audio from bytes to an AudioSegment object
    audio = AudioSegment.from_file(io.BytesIO(audio), format="mp3")

    # Exporting the audio to an MP3 file named "output.mp3"
    audio.export("output.mp3", format="mp3")

    return audio

In [ ]:
text = "The warm sunlight gently streamed through the windows, filling the room with a soft, golden glow, as laughter and cheerful conversation echoed amongst friends, creating an atmosphere of joy and contentment that uplifted everyone's spirits"
text_to_speech(text)

#### Text Summary

In [ ]:
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
def text_to_summary(text):
  summarized_text = summarizer(text, max_length=130, min_length=30, do_sample=False)
  return summarized_text[0]['summary_text']

In [ ]:
ARTICLE = """ Dear [Customer Name],

Thank you for reaching out to us at [Company Name]. I hope this email finds you well. I am writing in response to your inquiry dated [Date], regarding the status of your recent order with us, order number 12345.

Firstly, I would like to extend our sincerest apologies for any inconvenience you may have experienced in relation to this order. At [Company Name], we pride ourselves on ensuring our customer's satisfaction and it seems we have fallen short in this instance.

Upon reviewing your order details, I noticed that there was an unexpected delay in the shipment of your items due to an unforeseen stock issue with one of our suppliers. This has unfortunately impacted the delivery time frame of your order. I assure you that we are working diligently to resolve this matter as swiftly as possible.

To keep you informed, here are the steps we are taking:

Restocking: We are expediting the restocking process for the unavailable item(s). Our team is in constant communication with our supplier to ensure a speedy resolution.
Alternative Options: If you prefer not to wait for the restock, we are happy to offer you a substitute product or a different color/size of the same product, if available.
Shipping Update: Once the item is back in stock, we will prioritize your order for immediate shipping. We will also upgrade your shipping method to express delivery at no additional cost to you.
Compensation: As an apology for the inconvenience caused, we would like to offer you a [percentage] discount on your next purchase with us.
I understand that this situation may have caused a significant inconvenience, and for that, we are truly sorry. Your satisfaction is our top priority, and we are committed to making this right for you.

Additionally, I would like to offer you a direct line of communication with me for any further concerns or inquiries you may have. You can reach me at this email or directly at [Your Phone Number].

We value your business and appreciate your understanding in this matter. We look forward to resolving this issue to your satisfaction and hope to continue serving you in the future.

Thank you for your patience and understanding. Please feel free to reach out with any more questions, concerns, or feedback.

Warm regards,
"""
print(text_to_summary(ARTICLE))

An unexpected delay in the shipment of your items due to an unforeseen stock issue with one of our suppliers. This has unfortunately impacted the delivery time frame of your order. As an apology for the inconvenience caused, we would like to offer you a discount on your next purchase with us.


#### Reviews classification

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")
model = AutoModelForSequenceClassification.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")

tokenizer_config.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/669M [00:00<?, ?B/s]

In [ ]:
def classify_review(review):
  inputs = tokenizer(review, return_tensors="pt")

  with torch.no_grad():
    logits = model(**inputs).logits

  predicted_class_id = logits.argmax().item() + 1
  return predicted_class_id

In [ ]:
test = "I love it"
predicted_class_id = classify_review(test)

print(f"Sentiment class: {predicted_class_id} stars")

Sentiment class: 5 stars


#### Categorizer

In [ ]:
pipe = pipeline("text-classification", model="vineetsharma/customer-support-intent-albert")

config.json:   0%|          | 0.00/2.44k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/46.8M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/495 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.27M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/286 [00:00<?, ?B/s]

In [ ]:
def categorizer(text_to_classify):
  prediction = pipe(text_to_classify)
  return prediction[0]['label']

In [ ]:
categorizer('How do I reset my password?')

'recover_password'

#### structure 1

In [ ]:
email = """ I recently purchased the [Product Name] and have been mostly pleased with my experience. The product arrived on time, and the packaging was secure and environmentally friendly, which I appreciate.

The quality of [Product Name] is impressive; it's durable and functions exactly as advertised. I've been using it for several weeks now, and it's clear that it is well-made and reliable. The customer service team was also helpful when I had questions about the product features, providing prompt and courteous responses.

However, I've deducted one star because the assembly instructions were not as clear as I would have hoped, and it took me a while to set everything up correctly. Additionally, while the product works well, it does not integrate seamlessly with my existing [Related Product/System], which is something I think could be improved.

Overall, I'm very satisfied with my purchase and would recommend the [Product Name] to others looking for quality and reliability. It's not perfect, but it's certainly one of the better options available on the market.
"""

In [ ]:
email_summary = text_to_summary(translate_to_english(email))
sentiment = text_to_sentiment(email_summary)
email_audio = text_to_speech(email_summary)
category = categorizer(email_summary)

if category.isin(['review']):
  stars = classify_review(email_summary)

IndexError: list index out of range

In [ ]:
email_summary

NameError: name 'email_summary' is not defined